참고 url : https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

In [1]:
!pip install transformers
!pip install sentencepiece==0.1.91

     |████████████████████████████████| 3.1 MB 9.8 MB/s 
     |████████████████████████████████| 596 kB 83.5 MB/s 
     |████████████████████████████████| 895 kB 51.4 MB/s 
     |████████████████████████████████| 3.3 MB 58.9 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 10.0 MB/s 


In [ ]:
# Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

런타임 다시 시작 눌러야할 수 있음  
만일 tokenizer Nonetype 에러시 런타임 다시 시작

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# model.generate(pieces)
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

model_folder = '/content/drive/MyDrive/cakd3_colab/3rd_project/etri_et5'

# model = T5ForConditionalGeneration.from_pretrained(model_folder)
# tokenizer = T5Tokenizer.from_pretrained(model_folder)

model_name = 'KETI-AIR/ke-t5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/597 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/293M [00:00<?, ?B/s]

In [4]:
class CustomDataset:

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, padding='max_length', return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, padding='max_length', return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [5]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in tqdm(enumerate(loader, 0)):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            pass
            
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [6]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [12]:
!export CUDA_VISIBLE_DEVICES=0
!nvidia-smi

Tue Nov 23 05:53:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    33W / 250W |   1443MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
# Setting up the device for GPU usage
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [14]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(64128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(64128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo):

hyper-parameters

In [15]:
config = T5Config()
config.MAX_LEN = 1024
config.SUMMARY_LEN = 150 
config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 3        # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)

In [16]:
train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

In [17]:
import pandas as pd
train_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/extract_data/train_data.csv')[['document','label']]
validation_dataset = pd.read_csv('/content/drive/MyDrive/3차 프로젝트/dataset/extract_data/validation_data.csv')[['document','label']]

partial dataset


In [18]:
import numpy as np

train_dataset = train_dataset.sample(frac=1).reset_index(drop=True).iloc[:2000]
validation_dataset = validation_dataset.sample(frac=1).reset_index(drop=True).iloc[:500]

train

In [19]:
train_dataset.columns = ['ctext','text']
train_dataset.ctext = 'summarize: ' + train_dataset.ctext

training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

training_loader = DataLoader(training_set, **train_params)

for epoch in range(config.TRAIN_EPOCHS):
    print (1)
    train(epoch, tokenizer, model, device, training_loader, optimizer)

1


1it [00:00,  2.87it/s]

Epoch: 0, Loss:  45.02627944946289


501it [01:16,  6.50it/s]

Epoch: 0, Loss:  9.693105697631836


...

1it [00:00,  6.44it/s]

Epoch: 2, Loss:  7.499962329864502


501it [01:16,  6.52it/s]

Epoch: 2, Loss:  6.687759876251221


1000it [02:32,  6.55it/s]


valid(test)

In [20]:
validation_dataset.columns = ['ctext','text']
validation_dataset.ctext = 'summarize: ' + validation_dataset.ctext

val_set = CustomDataset(validation_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

val_loader = DataLoader(val_set, **val_params)

for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})

final_df

Completed 0
Completed 100
Completed 200


,Generated Text,Actual Text
0,原原原原原原原原原原 때문이었다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다...,29일 수원 광교비즈니스센터에서 경기도가 주최하고 경기도콘텐츠진흥원이 주관하는 'N...
1,原原原原原原原原原 때문이었다原 때문이었다 때문이었다 때문이었다 때문이었다 때문이었다...,민간제안펀드를 대폭 늘리고 최대 출자비 30%로 줄여 벤처투자의 마중물 역할을 하는...
2,原原原原原原原原原原原原原原原原原 때문이었다 눈물이 눈물이 주목된다 공개된다 공개된다...,태국 람차방항구에 정박 중 폭발사고가 난 고려해운 화물이 위험화물인가 승인 조치를 ...
3,原原原原原原原原原原 때문이었다 자네原山山 거래되고 거래되고 거래되고 거래되고 거래되...,오리지널 콘텐츠 제작을 강화하여 미디어 사업 성장을 위해 KT그룹과 디스커버리가 연...
4,原原 위메프原原原原原原原 한국경영자총협회 금통위 위원들 위원들 여러분이 마찬가지입니...,BNK경남은행은 24일 금융감독원 경남지원과 공동으로 '도서벽지 초등학교 무빙뱅크 ...
...,...,...
495,原原原原原原原原原原原 때문이었다 자네原kmib비서관는 자기자본이 고용과 공정거래고용...,문재인 대통령은 현장 소통을 통해 고용지표 악화 등 민생경제에 대한 현장의 위기감은...
496,原原原原原原原原 때문이었다 자네原 때문이었다 때문이었다 때문이었다 때문이었다 때문이...,2년 동안 쉬지않고 달려온 손흥민(27 토트넘 홋스퍼)이 지난 11일 이란전을 끝으...
497,原原原原原原原原原原原 댓글이 거래되고 거래되고 거래되고 거래되고 거래되고 거래되고 ...,"오는 2일 산업부가 경북 구미시와 대구광역시, 부산광역시 전북 익산시에 뿌리산업 특..."
498,原原原原原原原原原 때문이었다原 때문이었다 때문이었다 때문이었다 때문이었다原 때문이었...,한국에너지공단은기획재정부가 주관한 총 128개 공공기관을 대상으로 시행된 '2018...


In [21]:
final_df.loc[10, 'Generated Text']

'原原原原原原原原原原原原◆ 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다 공개된다포천선자지구의 위한 상고기타인은 단국대가고이.'

In [22]:
final_df.loc[10, 'Actual Text']

'아이탑 경진대회는 국민 정보통신기술 역량을 증진시키고자 만든 자격 경진대회로 올해는 3700명이 참여하면서 한국생산성본부(KPC)와 전자신문사, 한국정보처리학회, 한국디지털정책학회의 주최하에 21일 전국 39개 고사장에서 진행되었다.'

In [23]:
final_df.to_csv('/content/drive/MyDrive/cakd3_colab/3rd_project/ke-t5_small.csv')

In [ ]:
tokenizer.save_pretrained('./pretrained/')
model.save_pretrained('./pretrained/')